# Activity: Build a random forest model

## **Introduction**


As you're learning, random forests are popular statistical learning algorithms. Some of their primary benefits include reducing variance, bias, and the chance of overfitting.

This activity is a continuation of the project you began modeling with decision trees for an airline. Here, you will train, tune, and evaluate a random forest model using data from spreadsheet of survey responses from 129,880 customers. It includes data points such as class, flight distance, and inflight entertainment. Your random forest model will be used to predict whether a customer will be satisfied with their flight experience.

**Note:** Because this lab uses a real dataset, this notebook first requires exploratory data analysis, data cleaning, and other manipulations to prepare it for modeling.

## **Step 1: Imports** 


Import relevant Python libraries and modules, including `numpy` and `pandas`libraries for data processing; the `pickle` package to save the model; and the `sklearn` library, containing:
- The module `ensemble`, which has the function `RandomForestClassifier`
- The module `model_selection`, which has the functions `train_test_split`, `PredefinedSplit`, and `GridSearchCV` 
- The module `metrics`, which has the functions `f1_score`, `precision_score`, `recall_score`, and `accuracy_score`


In [34]:
# Import `numpy`, `pandas`, `pickle`, and `sklearn`.
# Import the relevant functions from `sklearn.ensemble`, `sklearn.model_selection`, and `sklearn.metrics`.

# Import `numpy`, `pandas`, `pickle`, and `sklearn`.
# Import the relevant functions from `sklearn.ensemble`, `sklearn.model_selection`, and `sklearn.metrics`.

 
import numpy as np
import pandas as pd

import pickle as pkl
 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, PredefinedSplit, GridSearchCV
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

As shown in this cell, the dataset has been automatically loaded in for you. You do not need to download the .csv file, or provide more code, in order to access the dataset and proceed with this lab. Please continue with this activity by completing the following instructions.

In [35]:
# RUN THIS CELL TO IMPORT YOUR DATA. 

### YOUR CODE HERE ###

air_data = pd.read_csv(r"C:\Users\saswa\Documents\GitHub\Python-For-Data-Analysis\Course-6\Data\module_4_data\Invistico_Airline.csv")

<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

The `read_csv()` function from the `pandas` library can be helpful here.
 
</details>

Now, you're ready to begin cleaning your data. 

## **Step 2: Data cleaning** 

To get a sense of the data, display the first 10 rows.

In [36]:
# Display first 10 rows.

air_data.head(10)


,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0
5,satisfied,Loyal Customer,30,Personal Travel,Eco,1894,0,0,0,3,...,2,2,5,4,5,5,4,2,0,0.0
6,satisfied,Loyal Customer,66,Personal Travel,Eco,227,0,0,0,3,...,5,5,5,0,5,5,5,3,17,15.0
7,satisfied,Loyal Customer,10,Personal Travel,Eco,1812,0,0,0,3,...,2,2,3,3,4,5,4,2,0,0.0
8,satisfied,Loyal Customer,56,Personal Travel,Business,73,0,0,0,3,...,5,4,4,0,1,5,4,4,0,0.0
9,satisfied,Loyal Customer,22,Personal Travel,Eco,1556,0,0,0,3,...,2,2,2,4,5,3,4,2,30,26.0


<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

The `head()` function from the `pandas` library can be helpful here.
 
</details>

Now, display the variable names and their data types. 

In [37]:
# Display variable names and types.

air_data.dtypes

satisfaction                          object
Customer Type                         object
Age                                    int64
Type of Travel                        object
Class                                 object
Flight Distance                        int64
Seat comfort                           int64
Departure/Arrival time convenient      int64
Food and drink                         int64
Gate location                          int64
Inflight wifi service                  int64
Inflight entertainment                 int64
Online support                         int64
Ease of Online booking                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Cleanliness                            int64
Online boarding                        int64
Departure Delay in Minutes             int64
Arrival Delay in Minutes             float64
dtype: obj

<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

DataFrames have an attribute that outputs variable names and data types in one result.
 
</details>

**Question:** What do you observe about the differences in data types among the variables included in the data?

The dataset contains three types of variables: **integer (int64), floating-point (float64), and categorical (object).**

**Observations on Data Types:**  

- **Categorical Variables (object type)**  
  The variables `satisfaction`, `Customer Type`, `Type of Travel`, and `Class` are stored as **objects**.  
  These represent categorical data and need to be **encoded** (e.g., one-hot encoding or label encoding) before being used in machine learning models.

- **Numerical Variables (int64 & float64)**  
  Most numerical features, such as `Age`, `Flight Distance`, and customer service ratings (`Seat comfort`, `Online support`, etc.), are stored as **int64** (whole numbers).  
  `Arrival Delay in Minutes` is stored as **float64**, while `Departure Delay in Minutes` is **int64**.  
  This suggests that `Arrival Delay in Minutes` might contain **fractional values** or **missing values** that were imputed.

These differences in data types highlight the need for appropriate **data preprocessing** before model training.


Next, to understand the size of the dataset, identify the number of rows and the number of columns.

In [38]:
# Identify the number of rows and the number of columns.

air_data.shape

(129880, 22)

<details>
  <summary><h4><strong>Hint 1</strong></h4></summary>

There is a method in the `pandas` library that outputs the number of rows and the number of columns in one result.

</details>

Now, check for missing values in the rows of the data. Start with .isna() to get Booleans indicating whether each value in the data is missing. Then, use .any(axis=1) to get Booleans indicating whether there are any missing values along the columns in each row. Finally, use .sum() to get the number of rows that contain missing values.

In [39]:
# Get Booleans to find missing values in data.
# Get Booleans to find missing values along columns.
# Get the number of rows that contain missing values.

air_data.isna().any(axis=1).sum()

np.int64(393)

**Question:** How many rows of data are missing values?

Given that the output is `np.int64(393)`, this means that **393 rows** in the dataset have at least one missing value.

Drop the rows with missing values. This is an important step in data cleaning, as it makes the data more useful for analysis and regression. Then, save the resulting pandas DataFrame in a variable named `air_data_subset`.

In [40]:
# Drop missing values.
# Save the DataFrame in variable `air_data_subset`.

air_data_subset = air_data.dropna(axis=0)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

The `dropna()` function is helpful here.
</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

The axis parameter passed in to this function should be set to 0 (if you want to drop rows containing missing values) or 1 (if you want to drop columns containing missing values).
</details>

Next, display the first 10 rows to examine the data subset.

In [41]:
# Display the first 10 rows.

air_data_subset.head(10)

,satisfaction,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,satisfied,Loyal Customer,65,Personal Travel,Eco,265,0,0,0,2,...,2,3,3,0,3,5,3,2,0,0.0
1,satisfied,Loyal Customer,47,Personal Travel,Business,2464,0,0,0,3,...,2,3,4,4,4,2,3,2,310,305.0
2,satisfied,Loyal Customer,15,Personal Travel,Eco,2138,0,0,0,3,...,2,2,3,3,4,4,4,2,0,0.0
3,satisfied,Loyal Customer,60,Personal Travel,Eco,623,0,0,0,3,...,3,1,1,0,1,4,1,3,0,0.0
4,satisfied,Loyal Customer,70,Personal Travel,Eco,354,0,0,0,3,...,4,2,2,0,2,4,2,5,0,0.0
5,satisfied,Loyal Customer,30,Personal Travel,Eco,1894,0,0,0,3,...,2,2,5,4,5,5,4,2,0,0.0
6,satisfied,Loyal Customer,66,Personal Travel,Eco,227,0,0,0,3,...,5,5,5,0,5,5,5,3,17,15.0
7,satisfied,Loyal Customer,10,Personal Travel,Eco,1812,0,0,0,3,...,2,2,3,3,4,5,4,2,0,0.0
8,satisfied,Loyal Customer,56,Personal Travel,Business,73,0,0,0,3,...,5,4,4,0,1,5,4,4,0,0.0
9,satisfied,Loyal Customer,22,Personal Travel,Eco,1556,0,0,0,3,...,2,2,2,4,5,3,4,2,30,26.0


Confirm that it does not contain any missing values.

In [42]:
# Count of missing values.

air_data_subset.isna().sum()

satisfaction                         0
Customer Type                        0
Age                                  0
Type of Travel                       0
Class                                0
Flight Distance                      0
Seat comfort                         0
Departure/Arrival time convenient    0
Food and drink                       0
Gate location                        0
Inflight wifi service                0
Inflight entertainment               0
Online support                       0
Ease of Online booking               0
On-board service                     0
Leg room service                     0
Baggage handling                     0
Checkin service                      0
Cleanliness                          0
Online boarding                      0
Departure Delay in Minutes           0
Arrival Delay in Minutes             0
dtype: int64

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can use the `.isna().sum()` to get the number of missing values for each variable.

</details>

Next, convert the categorical features to indicator (one-hot encoded) features. 

**Note:** The `drop_first` argument can be kept as default (`False`) during one-hot encoding for random forest models, so it does not need to be specified. Also, the target variable, `satisfaction`, does not need to be encoded and will be extracted in a later step.

In [43]:
# Convert categorical features to one-hot encoded features.


air_data_subset_dummies = pd.get_dummies(air_data_subset, 
                                         columns=
                                         ['Customer Type','Type of Travel','Class'])

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can use the `pd.get_dummies()` function to convert categorical variables to one-hot encoded variables.
</details>

**Question:** Why is it necessary to convert categorical data into dummy variables?**


It is necessary to convert categorical data into dummy variables because most machine learning models, including random forests, require numerical input to perform calculations.  

Categorical variables contain text-based values (e.g., "Business", "Economy" for `Class`), which models cannot directly process. One-hot encoding with `pd.get_dummies()` converts each category into a separate binary (0 or 1) column, making the data suitable for model training.  

For example, if `Class` has three categories (`Economy`, `Business`, `First`), one-hot encoding creates three new columns (`Class_Economy`, `Class_Business`, `Class_First`), where each row has a 1 in the corresponding category's column and 0s elsewhere.  

This transformation ensures categorical features are properly interpreted by the model without introducing incorrect ordinal relationships.


Next, display the first 10 rows to review the `air_data_subset_dummies`. 

In [44]:
# Display the first 10 rows.

air_data_subset_dummies.head(10)

,satisfaction,Age,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,...,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,Customer Type_Loyal Customer,Customer Type_disloyal Customer,Type of Travel_Business travel,Type of Travel_Personal Travel,Class_Business,Class_Eco,Class_Eco Plus
0,satisfied,65,265,0,0,0,2,2,4,2,...,2,0,0.0,True,False,False,True,False,True,False
1,satisfied,47,2464,0,0,0,3,0,2,2,...,2,310,305.0,True,False,False,True,True,False,False
2,satisfied,15,2138,0,0,0,3,2,0,2,...,2,0,0.0,True,False,False,True,False,True,False
3,satisfied,60,623,0,0,0,3,3,4,3,...,3,0,0.0,True,False,False,True,False,True,False
4,satisfied,70,354,0,0,0,3,4,3,4,...,5,0,0.0,True,False,False,True,False,True,False
5,satisfied,30,1894,0,0,0,3,2,0,2,...,2,0,0.0,True,False,False,True,False,True,False
6,satisfied,66,227,0,0,0,3,2,5,5,...,3,17,15.0,True,False,False,True,False,True,False
7,satisfied,10,1812,0,0,0,3,2,0,2,...,2,0,0.0,True,False,False,True,False,True,False
8,satisfied,56,73,0,0,0,3,5,3,5,...,4,0,0.0,True,False,False,True,True,False,False
9,satisfied,22,1556,0,0,0,3,2,0,2,...,2,30,26.0,True,False,False,True,False,True,False


Then, check the variables of air_data_subset_dummies.

In [45]:
# Display variables.

air_data_subset_dummies.dtypes

satisfaction                          object
Age                                    int64
Flight Distance                        int64
Seat comfort                           int64
Departure/Arrival time convenient      int64
Food and drink                         int64
Gate location                          int64
Inflight wifi service                  int64
Inflight entertainment                 int64
Online support                         int64
Ease of Online booking                 int64
On-board service                       int64
Leg room service                       int64
Baggage handling                       int64
Checkin service                        int64
Cleanliness                            int64
Online boarding                        int64
Departure Delay in Minutes             int64
Arrival Delay in Minutes             float64
Customer Type_Loyal Customer            bool
Customer Type_disloyal Customer         bool
Type of Travel_Business travel          bool
Type of Tr

**Question:** What changes do you observe after converting the string data to dummy variables?**



After converting the string (categorical) data to dummy variables, we observe the following changes:  

- **New Binary Columns:**  
  - Categorical variables (`Customer Type`, `Type of Travel`, and `Class`) have been replaced with multiple binary (0 or 1) columns, representing each category.  
  - For example, `Customer Type` is now split into `Customer Type_Loyal Customer` and `Customer Type_disloyal Customer`, where each row has a 1 in the corresponding category's column.  

- **Data Type Changes:**  
  - The newly created dummy variable columns are now of type `bool` instead of `object`, making them compatible with numerical processing in machine learning models.  

- **Original Categorical Variables Removed:**  
  - The original string-based categorical columns are no longer in the dataset, as they have been replaced by their corresponding dummy variable columns.  

- **Other Columns Remain Unchanged:**  
  - Numeric columns (`Age`, `Flight Distance`, `Seat Comfort`, etc.) remain in their original integer (`int64`) or float (`float64`) format.  

These changes allow machine learning models to process categorical data effectively without misinterpreting them as ordinal variables.


## **Step 3: Model building** 

The first step to building your model is separating the labels (y) from the features (X).

In [46]:
# Separate the dataset into labels (y) and features (X).

y = air_data_subset_dummies["satisfaction"]
X = air_data_subset_dummies.drop("satisfaction", axis=1)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Save the labels (the values in the `satisfaction` column) as `y`.

Save the features as `X`. 

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

To obtain the features, drop the `satisfaction` column from the DataFrame.

</details>

Once separated, split the data into train, validate, and test sets. 

In [47]:
# Separate into train, validate, test sets.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 0)


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `train_test_split()` function twice to create train/validate/test sets, passing in `random_state` for reproducible results. 

</details>

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Split `X`, `y` to get `X_train`, `X_test`, `y_train`, `y_test`. Set the `test_size` argument to the proportion of data points you want to select for testing. 

Split `X_train`, `y_train` to get `X_tr`, `X_val`, `y_tr`, `y_val`. Set the `test_size` argument to the proportion of data points you want to select for validation. 

</details>

### Tune the model

Now, fit and tune a random forest model with separate validation set. Begin by determining a set of hyperparameters for tuning the model using GridSearchCV.


In [48]:
# Determine set of hyperparameters.

cv_params = {'n_estimators' : [50,100], 
              'max_depth' : [10,50],        
              'min_samples_leaf' : [0.5,1], 
              'min_samples_split' : [0.001, 0.01],
              'max_features' : ["sqrt"], 
              'max_samples' : [.5,.9]}

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Create a dictionary `cv_params` that maps each hyperparameter name to a list of values. The GridSearch you conduct will set the hyperparameter to each possible value, as specified, and determine which value is optimal.

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

The main hyperparameters here include `'n_estimators', 'max_depth', 'min_samples_leaf', 'min_samples_split', 'max_features', and 'max_samples'`. These will be the keys in the dictionary `cv_params`.

</details>

Next, create a list of split indices.

In [49]:
# Create list of split indices.

split_index = [0 if x in X_val.index else -1 for x in X_train.index]
custom_split = PredefinedSplit(split_index)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use list comprehension, iterating over the indices of `X_train`. The list can consists of 0s to indicate data points that should be treated as validation data and -1s to indicate data points that should be treated as training data.

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

Use `PredfinedSplit()`, passing in `split_index`, saving the output as `custom_split`. This will serve as a custom split that will identify which data points from the train set should be treated as validation data during GridSearch.

</details>

Now, instantiate your model.

In [50]:
# Instantiate model.

rf = RandomForestClassifier(random_state=0)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use `RandomForestClassifier()`, specifying the `random_state` argument for reproducible results. This will help you instantiate a random forest model, `rf`.

</details>

Next, use GridSearchCV to search over the specified parameters.

In [51]:
# Search over specified parameters.


rf_val = GridSearchCV(rf, cv_params, cv=custom_split, refit='f1', n_jobs = -1, verbose = 1)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use `GridSearchCV()`, passing in `rf` and `cv_params` and specifying `cv` as `custom_split`. Additional arguments that you can specify include: `refit='f1', n_jobs = -1, verbose = 1`. 

</details>

Now, fit your model.

In [52]:
%%time

# Fit the model.

rf_val.fit(X_train, y_train)


Fitting 1 folds for each of 32 candidates, totalling 32 fits
CPU times: total: 4.34 s
Wall time: 11.3 s


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ..., -1, -1])),
             estimator=RandomForestClassifier(random_state=0), n_jobs=-1,
             param_grid={'max_depth': [10, 50], 'max_features': ['sqrt'],
                         'max_samples': [0.5, 0.9],
                         'min_samples_leaf': [0.5, 1],
                         'min_samples_split': [0.001, 0.01],
                         'n_estimators': [50, 100]},
             refit='f1', verbose=1)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `fit()` method to train the GridSearchCV model on `X_train` and `y_train`. 

</details>

<details>
<summary><h4><strong>Hint 2</strong></h4></summary>

Add the magic function `%%time` to keep track of the amount of time it takes to fit the model and display this information once execution has completed. Remember that this code must be the first line in the cell.

</details>

Finally, obtain the optimal parameters.

In [53]:
# Obtain optimal parameters.

rf_val.best_params_

{'max_depth': 50,
 'max_features': 'sqrt',
 'max_samples': 0.9,
 'min_samples_leaf': 1,
 'min_samples_split': 0.001,
 'n_estimators': 50}

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `best_params_` attribute to obtain the optimal values for the hyperparameters from the GridSearchCV model.

</details>

## **Step 4: Results and evaluation** 

Use the selected model to predict on your test data. Use the optimal parameters found via GridSearchCV.

In [54]:
# Use optimal parameters on GridSearchCV.

rf_opt = RandomForestClassifier(n_estimators = 50, max_depth = 50, 
                                min_samples_leaf = 1, min_samples_split = 0.001,
                                max_features="sqrt", max_samples = 0.9, random_state = 0)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use `RandomForestClassifier()`, specifying the `random_state` argument for reproducible results and passing in the optimal hyperparameters found in the previous step. To distinguish this from the previous random forest model, consider naming this variable `rf_opt`.

</details>

Once again, fit the optimal model.

In [55]:
# Fit the optimal model.

rf_opt.fit(X_train, y_train)

RandomForestClassifier(max_depth=50, max_samples=0.9, min_samples_split=0.001,
                       n_estimators=50, random_state=0)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Use the `fit()` method to train `rf_opt` on `X_train` and `y_train`.

</details>

And predict on the test set using the optimal model.

In [56]:
# Predict on test set.

y_pred = rf_opt.predict(X_test)

<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `predict()` function to make predictions on `X_test` using `rf_opt`. Save the predictions now (for example, as `y_pred`), to use them later for comparing to the true labels. 

</details>

### Obtain performance scores

First, get your precision score.

In [57]:
# Get precision score.

pc_test = precision_score(y_test, y_pred, pos_label = "satisfied")
print("The precision score is {pc:.3f}".format(pc = pc_test))

The precision score is 0.950


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `precision_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

Then, collect the recall score.

In [58]:
# Get recall score.

rc_test = recall_score(y_test, y_pred, pos_label = "satisfied")
print("The recall score is {rc:.3f}".format(rc = rc_test))

The recall score is 0.945


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `recall_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

Next, obtain your accuracy score.

In [59]:
# Get accuracy score.

ac_test = accuracy_score(y_test, y_pred)
print("The accuracy score is {ac:.3f}".format(ac = ac_test))

The accuracy score is 0.942


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `accuracy_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

Finally, collect your F1-score.

In [60]:
# Get F1 score.

f1_test = f1_score(y_test, y_pred, pos_label = "satisfied")
print("The F1 score is {f1:.3f}".format(f1 = f1_test))

The F1 score is 0.947


<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

You can call the `f1_score()` function from `sklearn.metrics`, passing in `y_test` and `y_pred` and specifying the `pos_label` argument as `"satisfied"`.
</details>

**Question:** How is the F1-score calculated?



The **F1-score** is a measure of a model's accuracy that considers both precision and recall. It is the **harmonic mean** of precision and recall, ensuring a balance between the two. The formula for the F1-score is:  

F1 = 2 × (Precision × Recall) / (Precision + Recall)

Where:  
- **Precision** = (True Positives) / (True Positives + False Positives)  
- **Recall** = (True Positives) / (True Positives + False Negatives)  

The F1-score ranges between **0 and 1**, where **1** represents a perfect model and **0** means poor performance. It is particularly useful when there is an **imbalance** in the dataset, ensuring that both precision and recall are considered in the model evaluation.


**Question:** What are the pros and cons of performing the model selection using test data instead of a separate validation dataset?




Performing model selection using test data instead of a separate validation dataset has both advantages and disadvantages.  

**Pros:**  
- Reduces the coding workload by eliminating the need for a separate validation set.  
- Shortens the data splitting process and simplifies the workflow.  
- Requires only a single evaluation on the test dataset instead of two separate evaluations (validation and test).  

**Cons:**  
- **Bias in Evaluation**: The model may produce overly optimistic results if the test set is used for both tuning and evaluation.  
- **Risk of Overfitting**: The model might be fine-tuned based on the test set, reducing its ability to generalize to truly unseen data.  
- **Data Leakage**: Using the test set for model selection means that it indirectly influences model development, which undermines its role as an independent evaluation dataset.  

**Best Practice:**  
A separate validation set or cross-validation should be used for model selection, ensuring that the test data remains untouched until the final performance assessment.


### Evaluate the model

Now that you have results, evaluate the model. 

**Question:** What are the four basic parameters for evaluating the performance of a classification model?


The four basic parameters for evaluating the performance of a classification model are:

- **True Positives (TP):** Cases where the model correctly predicts a positive outcome.  
- **True Negatives (TN):** Cases where the model correctly predicts a negative outcome.  
- **False Positives (FP):** Cases where the model incorrectly predicts a positive outcome when it is actually negative (also known as a Type I error).  
- **False Negatives (FN):** Cases where the model incorrectly predicts a negative outcome when it is actually positive (also known as a Type II error).  

These parameters are essential for calculating performance metrics such as accuracy, precision, recall, and F1-score, helping to assess the effectiveness of a classification model.


**Question:**  What do the four scores demonstrate about your model, and how do you calculate them?



**Precision:**  
- Measures the proportion of positive predictions that are actually correct.  
- Formula: `Precision = TP / (TP + FP)`  
- High precision means fewer false positives.  

**Recall (Sensitivity):**  
- Measures the model’s ability to correctly identify all actual positive cases.  
- Formula: `Recall = TP / (TP + FN)`  
- High recall means fewer false negatives.  

**Accuracy:**  
- Measures the overall correctness of predictions.  
- Formula: `Accuracy = (TP + TN) / (TP + TN + FP + FN)`  
- High accuracy indicates the model performs well on both positive and negative classes.  

**F1-score:**  
- The harmonic mean of precision and recall, balancing both metrics.  
- Formula: `F1-score = 2 × (Precision × Recall) / (Precision + Recall)`  
- Useful when both false positives and false negatives need to be minimized.  

Each metric provides unique insights into model performance. The choice of metric depends on the problem—recall is crucial in medical diagnoses (minimizing false negatives), while precision is more important in fraud detection (minimizing false positives).


Calculate the scores: precision score, recall score, accuracy score, F1 score.

In [61]:
# Precision score on test data set.

print("\nThe precision score is: {pc:.3f}".format(pc = pc_test), "for the test set,", "\nwhich means of all positive predictions,", "{pc_pct:.1f}% prediction are true positive.".format(pc_pct = pc_test * 100))


The precision score is: 0.950 for the test set, 
which means of all positive predictions, 95.0% prediction are true positive.


In [62]:
# Recall score on test data set.

print("\nThe recall score is: {rc:.3f}".format(rc = rc_test), "for the test set,", "\nwhich means of which means of all real positive cases in test set,", "{rc_pct:.1f}% are  predicted positive.".format(rc_pct = rc_test * 100))


The recall score is: 0.945 for the test set, 
which means of which means of all real positive cases in test set, 94.5% are  predicted positive.


In [63]:
# Accuracy score on test data set.

print("\nThe accuracy score is: {ac:.3f}".format(ac = ac_test), "for the test set,", "\nwhich means of all cases in test set,", "{ac_pct:.1f}% are predicted true positive or true negative.".format(ac_pct = ac_test * 100))


The accuracy score is: 0.942 for the test set, 
which means of all cases in test set, 94.2% are predicted true positive or true negative.


In [64]:
# F1 score on test data set.

print("\nThe F1 score is: {f1:.3f}".format(f1 = f1_test), "for the test set,", "\nwhich means the test set's harmonic mean is {f1_pct:.1f}%.".format(f1_pct = f1_test * 100)) 



The F1 score is: 0.947 for the test set, 
which means the test set's harmonic mean is 94.7%.


**Question:** How does this model perform based on the four scores?



The model performs well based on the four evaluation scores:

- **Precision (0.950)**: The model correctly identifies 95% of the positive predictions as actual positives, indicating a low false positive rate.
- **Recall (0.945)**: The model successfully captures 94.5% of the actual positive cases, suggesting minimal false negatives.
- **Accuracy (0.942)**: The model correctly predicts 94.2% of all instances, demonstrating strong overall performance.
- **F1-score (0.947)**: A balanced score between precision and recall, indicating the model effectively handles both false positives and false negatives.

**Conclusion:**  
With high precision, recall, accuracy, and an F1-score close to 1, the model demonstrates strong performance. However, further testing on unseen data is recommended to confirm its generalizability.


### Evaluate the model

Finally, create a table of results that you can use to evaluate the performace of your model.

In [65]:
# Create table of results.

table = pd.DataFrame({'Model': ["Tuned Decision Tree", "Tuned Random Forest"],
                        'F1':  [0.945422, f1_test],
                        'Recall': [0.935863, rc_test],
                        'Precision': [0.955197, pc_test],
                        'Accuracy': [0.940864, ac_test]
                      }
                    )
table

,Model,F1,Recall,Precision,Accuracy
0,Tuned Decision Tree,0.945422,0.935863,0.955197,0.940864
1,Tuned Random Forest,0.947306,0.944501,0.950128,0.942450



<details>
<summary><h4><strong>Hint 1</strong></h4></summary>

Build a table to compare the performance of the models. Create a DataFrame using the `pd.DataFrame()` function.

</details>

**Question:** How does the random forest model compare to the decision tree model you built in the previous lab?



The **Tuned Random Forest model** slightly outperforms the **Tuned Decision Tree model** in overall performance metrics:

- **F1-score**:  
  - Random Forest: **0.9473**  
  - Decision Tree: **0.9454**  
  - The Random Forest model has a slightly better balance between precision and recall.

- **Recall**:  
  - Random Forest: **0.9445**  
  - Decision Tree: **0.9359**  
  - The Random Forest model captures more actual positives compared to the Decision Tree.

- **Precision**:  
  - Random Forest: **0.9501**  
  - Decision Tree: **0.9552**  
  - The Decision Tree has a slightly higher precision, meaning fewer false positives.

- **Accuracy**:  
  - Random Forest: **0.9425**  
  - Decision Tree: **0.9409**  
  - The Random Forest model provides a small improvement in overall correctness.

**Conclusion:**  
The **Random Forest model** demonstrates slightly better overall performance, particularly in recall and F1-score. Since Random Forests use multiple trees, they tend to generalize better and reduce overfitting compared to a single Decision Tree. However, if computational efficiency is a concern, a well-tuned Decision Tree might still be a viable option.


## **Considerations**


**What are the key takeaways from this lab? Consider important steps when building a model, most effective approaches and tools, and overall results.**

### **Key Takeaways from this Lab**
- **Data exploration, cleaning, and encoding** are essential for preparing data before model building.
- **A separate validation set** should be used for tuning the model instead of the test set to ensure unbiased evaluation.
- **F1 score** is often a better metric than accuracy when false positives and false negatives carry different costs, as it balances **precision** and **recall**.
- **Random Forest models** typically perform better than Decision Trees due to their ensemble nature, reducing overfitting and improving generalization.


**What summary would you provide to stakeholders?**

### **Summary for Stakeholders**
- The **Random Forest model** achieved **94.2% accuracy** in predicting customer satisfaction, with **precision over 95%** and **recall around 94.5%**, ensuring a balanced and reliable performance.
- Compared to the **tuned Decision Tree model**, the **Random Forest model outperformed it in most key evaluation metrics**, making it the better choice for prediction.
- Since stakeholders are particularly interested in **the key factors affecting customer satisfaction**, insights from the **Random Forest feature importance analysis** will be shared.
- Additionally, **precision, recall, accuracy, and F1-score** will be presented to provide a comprehensive understanding of the model’s effectiveness.


### References

[What is the Difference Between Test and Validation Datasets?,  Jason Brownlee](https://machinelearningmastery.com/difference-test-validation-datasets/)

[Decision Trees and Random Forests Neil Liberman](https://towardsdatascience.com/decision-trees-and-random-forests-df0c3123f991)

**Congratulations!** You've completed this lab. However, you may not notice a green check mark next to this item on Coursera's platform. Please continue your progress regardless of the check mark. Just click on the "save" icon at the top of this notebook to ensure your work has been logged